Shopify Challenge

In [1]:
import pandas as pd

C:\Users\subha\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\subha\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\subha\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


Question 1

In [2]:
df = pd.read_csv('shopify_data.csv')

Exploratory Data Analysis

In [4]:
df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11


In [10]:
df['order_amount'].describe()

count      5000.000000
mean       3145.128000
std       41282.539349
min          90.000000
25%         163.000000
50%         284.000000
75%         390.000000
max      704000.000000
Name: order_amount, dtype: float64

In [11]:
df['total_items'].describe()

count    5000.00000
mean        8.78720
std       116.32032
min         1.00000
25%         1.00000
50%         2.00000
75%         3.00000
max      2000.00000
Name: total_items, dtype: float64

In [12]:
df.shape

(5000, 7)

In [14]:
df.isnull().sum()

order_id          0
shop_id           0
user_id           0
order_amount      0
total_items       0
payment_method    0
created_at        0
dtype: int64

Question 1 a

The reason why the calculation is wrong using the method described is that the number of items being sold in a single order is order is greater than 1. As can be seen above, the mean number of items being sold in a single order is 8.78 and hence naively calculating the AOV would lead to incorrect results. If we want to look at AOV then we would need to ensure that each order has only one item. This is done below.

In [15]:
df['average_order_amount'] = df['order_amount'] / df['total_items']

In [18]:
df['average_order_amount'].describe()

count     5000.000000
mean       387.742800
std       2441.963725
min         90.000000
25%        133.000000
50%        153.000000
75%        169.000000
max      25725.000000
Name: average_order_amount, dtype: float64

Once we take into account the average amount in an order instead of total, the average price drops to 387.7428. However, since we are calculating average let us take into account outliers since that would affect average calculation.


In [24]:
df.groupby(df['shop_id'])['average_order_amount'].mean().sort_values(ascending = False).head()


shop_id
78    25725.0
42      352.0
12      201.0
89      196.0
99      195.0
Name: average_order_amount, dtype: float64

From above, we can see that shop 78 is a outlier since it charges excessively higher than the other shops. Let us now calculate the mean without taking into account shop 78

In [26]:
df[df['shop_id'] != 78]['average_order_amount'].describe()

count    4954.000000
mean      152.475575
std        31.260218
min        90.000000
25%       132.000000
50%       153.000000
75%       168.000000
max       352.000000
Name: average_order_amount, dtype: float64

Once we remove shop 78, we get the mean to be 152.475 which is a more realistic value

In addition, calculating the average might not be the best practice since it can be skewed if there are outliers which can be difficult to remove and hence in this case median can be a good representation

Let us also check if there are any outliers in total_items

In [28]:
df['total_items'].value_counts()

2       1832
1       1830
3        941
4        293
5         77
2000      17
6          9
8          1
Name: total_items, dtype: int64

There are 17 orders with 2000 items being sold. Let us see if those could be removed

In [29]:
df[df['total_items'] == 2000]['shop_id'].unique()


array([42], dtype=int64)

All these orders are coming from shop 42. Let us investigate our results once we remove shop 42 as well.

In [31]:
df[(df['shop_id'] != 78) & (df['shop_id'] != 42)]['total_items'].describe()

count    4903.000000
mean        1.995717
std         0.982602
min         1.000000
25%         1.000000
50%         2.000000
75%         3.000000
max         8.000000
Name: total_items, dtype: float64

Now we see that the average total_items per order has decreased to about 2 which is more realistic. Now let us re-calculate our average amount in each order after removing shops 42 and 78 as outliers.

In [32]:
df[(df['shop_id'] != 78) & (df['shop_id'] != 42)]['order_amount'].describe()

count    4903.000000
mean      300.155823
std       155.941112
min        90.000000
25%       163.000000
50%       284.000000
75%       386.500000
max      1086.000000
Name: order_amount, dtype: float64

As we see above, the average order amounnt is about 300 and the average items in our order is about 2.

Calculating average might not always be the best idea since it can be skewed based on values and hence it is better to look at the median of the dataset (50%) as can be seen above as a metric for the dataset as a whole. The median order amount is 284 and the median total items is 2

As can additional metric for each order, we can even calculate the z-score and check if it lies without acceptable range of values

In [34]:
df['order_amount'].describe()

count      5000.000000
mean       3145.128000
std       41282.539349
min          90.000000
25%         163.000000
50%         284.000000
75%         390.000000
max      704000.000000
Name: order_amount, dtype: float64

In [36]:
df['z_score'] = (df['order_amount'] - 3145.13) / 41282.54
df['z_score'] = df['z_score']
df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at,average_order_amount,z_score
0,1,53,746,224,2,cash,2017-03-13 12:36:56,112.0,-0.070759
1,2,92,925,90,1,cash,2017-03-03 17:38:52,90.0,-0.074005
2,3,44,861,144,1,cash,2017-03-14 4:23:56,144.0,-0.072697
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37,156.0,-0.072407
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11,156.0,-0.072407


choosing a threshold of 0.1 for z-score

In [40]:
df[abs(df['z_score']) > 0.1].head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at,average_order_amount,z_score
15,16,42,607,704000,2000,credit_card,2017-03-07 4:00:00,352.0,16.977029
60,61,42,607,704000,2000,credit_card,2017-03-04 4:00:00,352.0,16.977029
160,161,78,990,25725,1,credit_card,2017-03-12 5:56:57,25725.0,0.546959
490,491,78,936,51450,2,debit,2017-03-26 17:08:19,25725.0,1.170104
493,494,78,983,51450,2,cash,2017-03-16 21:39:35,25725.0,1.170104


As can be seen all the above outliers are from shops 42 and 78 which we identified in our above analysis as well.

As a result, for new metrics we can use median values for the entire dataset and z-scores for each order.

Question 2

SELECT  COUNT(ShipperName) OrdersShipped FROM( SELECT * FROM Orders ord LEFT JOIN Shippers shipper ON shipper.ShipperID = ord.ShipperID) GROUP BY ShipperName HAVING ShipperName = 'Speedy Express'

Answer is 54


SELECT LastName FROM(
(SELECT EmployeeID, COUNT(*) NumOrders FROM Orders
GROUP BY EmployeeID) emp
LEFT JOIN (SELECT EmployeeID, LastName FROM Employees) info
ON emp.EmployeeID = info.EmployeeID)
ORDER BY NumOrders DESC
LIMIT 1

Answer is Peacock

SELECT ProductName FROM (
SELECT ProductId, Country, SUM(Quantity) TotalQuantity FROM OrderDetails ord
LEFT JOIN (
SELECT o.OrderID, o.CustomerID, c.Country FROM Orders o
LEFT JOIN (SELECT CustomerId, Country FROM Customers) c
ON o.CustomerID = c.CustomerID) cinfo
ON ord.OrderID = cinfo.OrderID
GROUP BY Country, ProductID
HAVING Country = 'Germany') germany
LEFT JOIN (SELECT ProductId, ProductName FROM Products) prod
ON germany.ProductId = prod.ProductId
ORDER BY TotalQuantity DESC
LIMIT 1

Answer is Boston Crab Meat
